In [2]:
# import chess
import chess.svg
import time 
import log
import minimax
from treelib import Node, Tree
from IPython.display import SVG, display,HTML,clear_output
import os
import time

In [ ]:
#######################
# Initializations
#######################
# Get Board
board = chess.Board() 
# Default player is X
playerType = 'X'
# File handler 
player = ''
# Keep track of seconds passed in a game
secondsPassed = 0
# Checks if time expired
timeExpired = False
# Records moves that I have made
movesIMade = 0
# Time to sleep in-between moves
sleepTime = 0
# Default positions for player x and player y
currentPiecePositions = {'n':chess.C8,'k':chess.E8,'K':chess.E1,'R':chess.H1,'N':chess.G1}
####################################################################### 
# setupBoard
# Default board setup
#######################################################################
def setupBoard(board):
    board.clear_board()
    board.set_piece_at(chess.E1,chess.Piece(chess.KING,True))
    board.set_piece_at(chess.H1,chess.Piece(chess.ROOK,True))
    board.set_piece_at(chess.G1,chess.Piece(chess.KNIGHT,True) )
    board.set_piece_at(chess.E8,chess.Piece(chess.KING, False))
    board.set_piece_at(chess.C8,chess.Piece(chess.KNIGHT,False))
####################################################################### 
# GameOver
# Takes in "winner" and "comments". Winner will be X or Y or "TIED". 
# It will display to the # user which one won or whether they are tied 
# and displays the comments as well.
#######################################################################
def GameOver(winner,comments):
    declareWinner = ""
    if (winner == 'X' or winner == 'Y'):
        declareWinner = "Player "+str(winner)+" has won!"
    else:
        declareWinner = " TIED "
    txt = "****************************************\n"\
          "************** GAME OVER ***************\n"\
          "* "+str(declareWinner)+"      *\n"\
          "* Reason: "+str(comments)+"            *\n"\
          "****************************************\n"
    print(txt)
####################################################################### 
# isGameOver
# Takes in "board" and "playerType" to check if there is a checkmate,
# stalemate, or if the board is in check.
# If any of that is true, it calls the GameOver function
#######################################################################
def isGameOver(board,playerType):
    if (board.is_checkmate() or board.is_stalemate()):
        if (board.is_check()):
            if(playerType == 'X'):
                GameOver("X","Player Y is in checkmate")
            else:
                GameOver("Y","Player X is in checkmate")
        else:
            GameOver("T","Stalemate")
        return True
    
    return False
####################################################################### 
# getSquareIndexByString
# Gets the index of the square name by going through python chess
# library' SQUARE_NAMES array and checking which one matches the string
#######################################################################
def getSquareIndexByString(str):
    index = 0
    for val in chess.SQUARE_NAMES:
        if val == str:
            return index
        index = index+1
    return -1
####################################################################### 
# displayBoard
# Displays the SVG board image of python's chess library
#######################################################################
def displayBoard(board):
    clear_output()
    boardImage = chess.svg.board(board)
    display(SVG(boardImage))
####################################################################### 
# getMovesByLineString
# Extracts the line string and interprets the move based on it and
# then goes ahead and makes that move on the chess board
#######################################################################
def getMovesByLineString(line):
    splitLine = line.split(':')
    moves = {'move': splitLine[2].strip()}
    # get the index value for the square
    moves['squareIndex'] = getSquareIndexByString(moves['move'])
    if(playerType == 'X'):
        moves['piece'] = splitLine[1].lower()
    else:
        moves['piece'] = splitLine[1]
    moves['pieceObject'] = chess.Piece.from_symbol(moves['piece'])
    return moves
####################################################################### 
# play
# This function is called when it's my turn to play. It gets the next
# move based on the minimax tree that is setup. After it gets the
# move, it makes the change on the board and records it in the file
# respectively. 
#######################################################################
def play(turn):
    global sleepTime
    time.sleep(sleepTime)
    tree = minimax.miniMaxTree(board,playerType)
    brd = tree.get_node(tree.root).data
    #for node in tree.children('currentBoard'):
    #    brd = node.data
    move = brd.peek()
    board.push(move)
    newSquare = str(move.uci()[2:4].strip())
    piece = str(board.piece_at(getSquareIndexByString(newSquare)))
    print(piece)
    displayBoard(board) 
    # If player is X, it needs to be capitalize
    # according to python chess' algorithm
    if (playerType == 'X'):
        piece = piece.capitalize()
    else:
        piece = piece.lower()
    # display the board to the user
    displayBoard(board)
    # keeps track of moves made
    global movesIMade
    movesIMade = movesIMade + 1
    player.write(turn,str(piece).capitalize(),newSquare,timeExpired)

    
#############################################
# PROGRAM STARTS HERE
#############################################
# Prompts user for player x or y
playerType = input("Player? (x or y)")
playerType = playerType.capitalize()
# Opens the appropriate file
if (playerType == 'X'):
    player = log.FileHandler('X')
else:
    player = log.FileHandler('Y')
    
# Setup the board
setupBoard(board)
# turns. Begins at 1.
turn = 1
#Gameover begins at false
gameover = False

# Player X starts move
if (playerType == 'X'):
    play(turn)
    turn = turn+1
    
##################################################
# The game will keep going until I have made 
# more than 100 moves or if there is no checkmate,
# stalemate, or tie.
##################################################
while(movesIMade <= 100 and not gameover):
    # Begins by waiting for next player to make move
    print("WAITING for player to make move....")
    timeExpired = False 
    # Start timer
    startTime = time.time()
    endTime = startTime
    secondsPassed = 0
    # Check if game is over
    gameover = isGameOver(board,playerType)
    # While game is not over and it has not passed 10 seconds
    # then it will wait for the next user. Once the next user
    # makes a move, it will break away from the while loop and get
    # that move.
    while(True and not gameover and secondsPassed <=10):
        endTime = time.time()
        # Record seconds passed
        secondsPassed = endTime-startTime
        wait = player.waitOpponent()
        # Wait until user makes a move
        if wait: 
            pass
        else:
            break 
    # check if time expired
    if (secondsPassed > 10):
            timeExpired = True
    # check if gameover
    if (not gameover):
        # Get the player's move
        m = player.getPreviousMove()
        # Get string
        playerMoves = getMovesByLineString(m)
        
        # If time has not expired then get the moves of the other player
        # and check if it results in a game over
        if(not timeExpired):
            movePiece = chess.Move(currentPiecePositions[playerMoves['piece']],playerMoves['squareIndex'])
            board.push(movePiece)
            currentPiecePositions[playerMoves['piece']] = getSquareIndexByString(playerMoves['move'])
            gameover = isGameOver(board,playerType)
        else: # if time expired then it will now be my turn. 
            board.turn = not board.turn
        # Turn now increased by 1
        turn=turn+1
        # Now my turn to play
        play(turn)
        turn = turn+1
# Show maximum 100 moves achieved if I made more than 100 moves
if (movesIMade == 101):
    print("Maximum 100 moves! Game is a TIE")
